## Introduction

We're going to make a neat looking plot in Altair: plotting the low-high values of a stock as area.

**The Problem**: "I want a graph that shows the high and low values of a particular stock."

Let's import all the stuff we'll need and begin.  We'll use ``SBUX`` stock data from 2021 from Yahoo! Finance.

(As of today, you can download the stock data [here](https://finance.yahoo.com/quote/SBUX/history?period1=1484956800&period2=1642723200&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true).  If this link doesn't work, any stock data should work similarly if it has a "low" and "high".)

In [52]:
import altair as alt
import pandas as pd

In [143]:
# Get the data, get the columns we'd like, plot the data with matplotlib.
df = pd.read_csv("../data/sbux.csv", parse_dates=["Date"])

# We'll keep "Open" just in case we need it later.
cols_to_keep = ["Date", "Open", "High", "Low"]
df = df[cols_to_keep]

# Only use 2020.
df = df[df["Date"].dt.year == 2020]

print(df.head(3))

          Date       Open       High        Low
741 2020-01-02  88.120003  89.349998  88.050003
742 2020-01-03  88.660004  89.099998  88.099998
743 2020-01-06  88.110001  88.410004  87.470001


## Plotting the Area in Altair

In [96]:
chart_highlow = (
    alt.Chart(df)
    .mark_area()
    .encode(
        x="Date:T", y=alt.Y("Low:Q", scale=alt.Scale(zero=False)), y2=alt.Y2("High:Q")
    )
)
chart_highlow

alt.Chart(...)

Okay, not too shabby, but this does look a bit jagged.

Maybe we can use Pandas' excellent ``resample`` method to take a weekly average.  This method is quite useful in translating from one time unit to another.  Check out the things we can resample by [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects).

We'll use ``W-MON`` which is a Week, starting on Monday.

(_Note: The following is sort of strange-looking at first, because we're setting and unsetting an index.  The idea is that ``resample`` will resample on the index (which we want to be the Date column) but Altair requires each column its using to be a non-index object.  This means we have to set the index to be the date, resample, and then reset the index for Altair._)

In [113]:
df_resampled = df.set_index("Date").resample("W-MON").mean()

chart_highlow = (
    alt.Chart(df_resampled.reset_index())
    .mark_area()
    .encode(
        x="Date:T", y=alt.Y("Low:Q", scale=alt.Scale(zero=False)), y2=alt.Y2("High:Q")
    )
)

chart_highlow

alt.Chart(...)

Looking pretty okay!  Little bit less jagged.  What if we wanted to also plot, say, the mean Opening value per week of the stock on this plot?  Maybe as a dotted line between these values?  Let's try it out.

In [126]:
df_resampled = df.set_index("Date").resample("W-MON").mean()

chart_highlow = (
    alt.Chart(df_resampled.reset_index())
    .mark_area(opacity=0.6)
    .encode(
        x="Date:T", y=alt.Y("Low:Q", scale=alt.Scale(zero=False)), y2=alt.Y2("High:Q")
    )
)

chart_opening = (
    alt.Chart(df_resampled.reset_index())
    .mark_line(opacity=0.8, strokeDash=[3, 1])
    .encode(
        x="Date:T",
        y=alt.Y("Open:Q", scale=alt.Scale(zero=False)),
        color=alt.value("red"),
    )
)

alt.layer(chart_highlow, chart_opening).interactive()

alt.LayerChart(...)

Neat.  Let's zoom in a bit and check out what it looks like.

In [141]:
df_resampled = df.set_index("Date").resample("W-MON").mean()
df_resampled_trimmed = df_resampled.iloc[:8, :]

chart_highlow = (
    alt.Chart(df_resampled_trimmed.reset_index())
    .mark_area(opacity=0.6)
    .encode(
        x="Date:T", y=alt.Y("Low:Q", scale=alt.Scale(zero=False)), y2=alt.Y2("High:Q")
    )
)

chart_opening = (
    alt.Chart(df_resampled_trimmed.reset_index())
    .mark_line(opacity=0.8, strokeDash=[3, 1])
    .encode(
        x="Date:T",
        y=alt.Y("Open:Q", scale=alt.Scale(zero=False)),
        color=alt.value("red"),
    )
)

alt.layer(chart_highlow, chart_opening)

alt.LayerChart(...)